In [5]:
import pandas as pd
import os
import glob

In [8]:
results_path = "/home/schmied.christopher/FMP_Docs/Projects/eu_os_ecbl_qc/results/"

In [ ]:
status_file = pd.read_csv(os.path.join(results_path, "Processing_Status.csv"))

# Check USC

In [ ]:
# Get USC files that are marked for processing
status_file_usc = status_file[status_file["source"] == "USC"]
status_file_usc_processed = status_file_usc[status_file_usc["process_folder"] == True]

In [16]:
status_file_usc_processed

,source,plate,batch,plate_name,replicate_number,plate_map_name,batch_date,batch_date_str,raw_file,process_folder,processed_file_exists,norm_file_exists,reduced_file_exists
1109,USC,C1157R1,240613,C1157,R1,C1157_R1,2024-06-13,2024-06-13,True,True,True,True,True
1111,USC,C1157R2,250509,C1157,R2,C1157_R2,2025-05-09,2025-05-09,True,True,True,True,True
1113,USC,C1157R3,250430,C1157,R3,C1157_R3,2025-04-30,2025-04-30,True,True,True,True,True
1115,USC,C1157R4,250507,C1157,R4,C1157_R4,2025-05-07,2025-05-07,True,True,True,True,True
1116,USC,C1158R1,240613,C1158,R1,C1158_R1,2024-06-13,2024-06-13,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,USC,USC02R1,240726,USC02,R1,USC02_R1,2024-07-26,2024-07-26,True,True,True,True,False
1442,USC,USC03R1,240927,USC03,R1,USC03_R1,2024-09-27,2024-09-27,True,True,True,True,False
1443,USC,USC04R1,241004,USC04,R1,USC04_R1,2024-10-04,2024-10-04,True,True,True,True,False
1444,USC,USC05R1,241016,USC05,R1,USC05_R1,2024-10-16,2024-10-16,True,True,True,True,False


In [34]:
def check_plate_id_match(array_1, array_2):
    set1 = set(array_1)
    set2 = set(array_2)

    if set1 == set2:
        print("Plate IDs match exactly")
    else:
        print("Plate IDs do NOT match")

        print("Missing in arr2:", sorted(set1 - set2))
        print("Extra in arr2:", sorted(set2 - set1))

In [35]:
# Check if all expected plate IDs are present in the processed files
# USC plate ID range: C1157 - C1229
expected_plates_usc = [f"C{i}" for i in range(1157, 1230)]

print("USC number of expected plates: " + str(len(expected_plates_usc)))

# TODO: Check if all plate IDs are present
# TODO: Check if all processed files have 4 replicates
processed_plates_usc =status_file_usc_processed["plate_name"].unique()

remove_control_plates = {"USC01", "USC02", "USC03", "USC04", "USC05", "USC06"}
processed_plates_usc_filtered = [x for x in processed_plates_usc if x not in remove_control_plates ]

print("USC number of processed plates: " + str(len(processed_plates_usc_filtered)))

check_plate_id_match(expected_plates_usc, processed_plates_usc_filtered)


USC number of expected plates: 73
USC number of processed plates: 73
Plate IDs match exactly


In [41]:
# Check if all expected plates have 4 replicates
replicate_counts = status_file_usc_processed.groupby("plate_name")["replicate_number"].nunique()
plates_with_4_replicates = replicate_counts[replicate_counts == 4].index.tolist()
print("Number of plates with 4 replicates: " + str(len(plates_with_4_replicates)))  

print("Checking if plates with 4 replicates match expected plates...")
check_plate_id_match(expected_plates_usc, plates_with_4_replicates)
plates_with_less_than_4_replicates = replicate_counts[replicate_counts < 4].index.tolist()
print("Plates with less than 4 replicates: " + str(plates_with_less_than_4_replicates))

Number of plates with 4 replicates: 73
Checking if plates with 4 replicates match expected plates...
Plate IDs match exactly
Plates with less than 4 replicates: ['USC01', 'USC02', 'USC03', 'USC04', 'USC05', 'USC06']


In [43]:
print("Checking if plates with less than 4 replicates match expected plates...")
check_plate_id_match(expected_plates_usc, plates_with_less_than_4_replicates)

Checking if plates with less than 4 replicates match expected plates...
Plate IDs do NOT match
Missing in arr2: ['C1157', 'C1158', 'C1159', 'C1160', 'C1161', 'C1162', 'C1163', 'C1164', 'C1165', 'C1166', 'C1167', 'C1168', 'C1169', 'C1170', 'C1171', 'C1172', 'C1173', 'C1174', 'C1175', 'C1176', 'C1177', 'C1178', 'C1179', 'C1180', 'C1181', 'C1182', 'C1183', 'C1184', 'C1185', 'C1186', 'C1187', 'C1188', 'C1189', 'C1190', 'C1191', 'C1192', 'C1193', 'C1194', 'C1195', 'C1196', 'C1197', 'C1198', 'C1199', 'C1200', 'C1201', 'C1202', 'C1203', 'C1204', 'C1205', 'C1206', 'C1207', 'C1208', 'C1209', 'C1210', 'C1211', 'C1212', 'C1213', 'C1214', 'C1215', 'C1216', 'C1217', 'C1218', 'C1219', 'C1220', 'C1221', 'C1222', 'C1223', 'C1224', 'C1225', 'C1226', 'C1227', 'C1228', 'C1229']
Extra in arr2: ['USC01', 'USC02', 'USC03', 'USC04', 'USC05', 'USC06']
